RAG - Loader(UpstageDocumentParseLoader)

In [ ]:
# 기본 LangChain 라이브러리
!pip install langchain langchain-core langchain-community

# LLM 연결을 위한 라이브러리 (Claude 사용)
!pip install langchain-anthropic

# 임베딩 및 벡터 데이터베이스
!pip install sentence-transformers faiss-cpu

# ReAct 에이전트 관련 라이브러리
!pip install langchain-experimental

# 환경 변수 관리
!pip install python-dotenv

# Upstage 문서 파서 (기존 코드에서 사용)
!pip install langchain-upstage

# HuggingFace 임베딩 (기존 코드에서 사용)
!pip install langchain-huggingface

In [ ]:
from langchain_upstage import UpstageDocumentParseLoader
from dotenv import load_dotenv
import os


# .env 파일 로드
load_dotenv()

file_path = "./test_modified.pdf"

loader = UpstageDocumentParseLoader(
    file_path,
    split="page",  # 페이지별로 분할
    output_format="markdown",  # 텍스트 형태로 출력
    ocr="auto",  # 자동 OCR 사용
    coordinates=True,  # 좌표 정보 포함
)

docs = loader.load()

In [ ]:
# print(docs[21].page_content)
# print(docs[0])

Rag - Chunking

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

#  문서 자르기 (AI가 읽기 쉬운 크기로 쪼개기)
splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=300)

# 문서 분할 실행
docs_splitter = splitter.split_documents(docs)

In [9]:
!pip install sentence_transformers

  Using cached tokenizers-0.21.2-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
Using cached tokenizers-0.21.2-cp39-abi3-macosx_11_0_arm64.whl (2.7 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-upstage 0.6.0 requires tokenizers<0.20.0,>=0.19.1, but you have tokenizers 0.21.2 which is incompatible.


Rag - Embedding

In [10]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.embeddings.base import Embeddings
from sentence_transformers import SentenceTransformer
from typing import List


# 기존 임베딩 그대로 사용
hf_embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large-instruct",
    model_kwargs={"device": "mps"},
    encode_kwargs={"normalize_embeddings": True},
)

Rag - Vector Store

In [11]:
# RAG - Vector Store
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(
    documents=docs_splitter,
    embedding=hf_embeddings,  # 커스텀 임베딩 사용
)

Rag - retriever

In [42]:
# RAG - retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5},
)

retriever = vectorstore.as_retriever(
    search_type="similarity",  # 유사도 검색
    search_kwargs={"k": 5},  # 상위 5개 결과 반환
)

Rag- retriever to tool

In [ ]:
from langchain.agents import AgentType, initialize_agent
from langchain.tools import Tool
from langchain_core.prompts import ChatPromptTemplate

# 필요한 임포트 추가
from langchain_core.prompts import MessagesPlaceholder
from langchain_anthropic import ChatAnthropic  # Claude 사용 시
# from langchain_aws import BedrockChat  # AWS Bedrock 사용 시

In [79]:

# Retriever를 직접 Tool로 래핑하는 함수
def search_documents(query: str) -> str:
    """문서 검색 도구 함수"""
    # 관련 문서 검색
    docs = retriever.invoke(query)
    
    # 검색 결과가 없는 경우
    if not docs:
        return "검색 결과가 없습니다."
    
    # 검색 결과 포맷팅
    formatted_results = []
    for i, doc in enumerate(docs, 1):
        # 각 문서 내용 추가
        formatted_results.append(f"[문서 {i}]\n{doc.page_content}\n")
    
    return "\n".join(formatted_results)

# Tool 객체 생성
document_search_tool = Tool(
    name="document_search",
    description="문서 내용을 검색할 때 사용하는 도구입니다. 제공된 PDF 문서 내에서 관련 정보를 찾아 제공합니다.",
    func=search_documents
)

# 에이전트 생성에 필요한 Tool 리스트
tools = [document_search_tool]

# LLM 설정
llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0)


# 에이전트 프롬프트 템플릿 생성
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", """너는 친절한 한국어 AI 비서야. 사용자의 질문에 답변하기 위해 문서 검색 도구를 사용할 수 있어.
    
    가장 중요한 규칙: 절대로 문서에 없는 내용을 답변에 포함하지 마세요. 
    문서에서 관련 정보를 찾지 못했다면 "죄송합니다만, 문서에서 해당 정보를 찾을 수 없습니다."라고만 답변하세요.
    
    항상 다음 과정을 따라야 해:
    1. 사용자의 질문을 분석해서 문서 검색이 필요한지 판단해
    2. 필요하다면 document_search 도구를 사용해 관련 정보를 찾아
    3. 검색 결과를 기반으로 한국어로 답변을 작성해
    4. 검색 결과에 관련 정보가 없다면 "죄송합니다만, 문서에서 해당 정보를 찾을 수 없습니다."라고 답변해
    5. 절대로 너의 일반 지식을 사용하여 문서에 없는 내용을 답변하지 마
    
    항상 한국어로 답변하고, 정확한 정보만 제공하도록 해.
    """),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# 에이전트 초기화
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,  # 대화형 에이전트 사용
    verbose=True,  # 실행 과정 출력
    max_iterations=3,  # 최대 반복 횟수
    handle_parsing_errors=True,  # 파싱 오류 처리
    early_stopping_method="generate",  # 조기 종료 방법
    prompt=agent_prompt  # 커스텀 프롬프트 사용
)

# 에이전트 실행 함수
def run_agent_with_history(query, chat_history=[]):
    """에이전트를 실행하여 질문에 답변"""
    try:
        response = agent.invoke({
            "input": query,
            "chat_history": chat_history
        })
        # 대화 히스토리 업데이트를 위한 출력 형식 맞추기
        return response["output"]
    except Exception as e:
        return f"에이전트 실행 중 오류 발생: {str(e)}"

# 대화 히스토리 관리
agent_chat_history = []

# 대화형 인터페이스
def interactive_agent_chat():
    """대화형 에이전트 인터페이스"""
    print("=" * 60)
    print("🤖 RAG 에이전트 시작")
    print("💡 '종료' 입력 시 대화를 끝냅니다.")
    print("=" * 60)

    try:
        while True:
            try:
                user_input = input("\n🙋 사용자: ").strip()

                if not user_input:
                    print("❗ 메시지를 입력해주세요.")
                    continue

                if user_input.lower() == "종료":
                    print("👋 채팅을 종료합니다!")
                    break

                print("🤖 AI: ", end="")
                response = run_agent_with_history(user_input, agent_chat_history)
                print(response)

                # 에이전트 히스토리 업데이트
                agent_chat_history.append({"role": "user", "content": user_input})
                agent_chat_history.append({"role": "assistant", "content": response})

            except KeyboardInterrupt:
                print("\n채팅을 종료합니다!")
                break

    finally:
        # 히스토리 초기화
        agent_chat_history.clear()

# 메인 실행 코드
if __name__ == "__main__":
    interactive_agent_chat()

🤖 RAG 에이전트 시작
💡 '종료' 입력 시 대화를 끝냅니다.
🤖 AI: 

> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "죄송합니다만, 제가 가진 정보로는 특정 과자를 추천해드리기 어렵습니다. 과자 추천은 개인의 취향과 선호도에 따라 크게 달라질 수 있기 때문입니다. 하지만 일반적으로 인기 있는 과자 몇 가지를 말씀드릴 수 있습니다:

1. 초코파이
2. 새우깡
3. 꼬깔콘
4. 오레오
5. 프링글스

이 외에도 다양한 맛있는 과자들이 있습니다. 개인의 취향에 따라 달콤한 과자, 짭짤한 과자, 바삭한 과자 등을 선택하실 수 있습니다. 또한 최근에는 건강에 좋은 재료를 사용한 과자들도 많이 출시되고 있으니 참고하시면 좋을 것 같습니다."
}
```

> Finished chain.
죄송합니다만, 제가 가진 정보로는 특정 과자를 추천해드리기 어렵습니다. 과자 추천은 개인의 취향과 선호도에 따라 크게 달라질 수 있기 때문입니다. 하지만 일반적으로 인기 있는 과자 몇 가지를 말씀드릴 수 있습니다:

1. 초코파이
2. 새우깡
3. 꼬깔콘
4. 오레오
5. 프링글스

이 외에도 다양한 맛있는 과자들이 있습니다. 개인의 취향에 따라 달콤한 과자, 짭짤한 과자, 바삭한 과자 등을 선택하실 수 있습니다. 또한 최근에는 건강에 좋은 재료를 사용한 과자들도 많이 출시되고 있으니 참고하시면 좋을 것 같습니다.
🤖 AI: 

> Entering new AgentExecutor chain...
아주대학교 건축학과의 수시 모집 인원을 확인하기 위해 문서를 검색해보겠습니다.

```json
{
    "action": "document_search",
    "action_input": "아주대학교 건축학과 수시 모집 인원"
}
```
Observation: [문서 1]
2026학년도 # 아주대학교 입학전형안내 본 입학전형 시행계획은 관계 법령 개정, 대